In [ ]:
import os
from __future__ import print_function

master = '--master local[1]'
#master = '--master spark://apachespark-master-2-1-0:7077'
#jars = '--jars /root/lib/jpmml-sparkml-1.2-SNAPSHOT.jar'
jars = ''
packages = '--packages org.jpmml:jpmml-sparkml:1.2.2'

py_files = '--py-files /root/lib/jpmml.py'

os.environ['PYSPARK_SUBMIT_ARGS'] = master \
  + ' ' + jars \
  + ' ' + packages \
  + ' ' + py_files \
  + ' ' + 'pyspark-shell'

print(os.environ['PYSPARK_SUBMIT_ARGS'])

In [ ]:
from pyspark.ml import Pipeline

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler


In [ ]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [14]:
import pandas as pd
from sklearn import datasets
iris = datasets.load_iris()

df = pd.DataFrame(iris.data, columns=iris.feature_names)

df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
df['species'] = pd.factorize(df['species'])[0]

df.columns = ["sepal_length(cm)", \
              "sepal_width(cm)", \
              "petal_length(cm)", \
              "petal_width(cm)", \
              "species"]

df_iris = sqlContext.createDataFrame(df)

df_iris.printSchema()

df_iris.show(5)

#train = df_iris

train, test = df_iris.randomSplit([0.7, 0.3])

train.count()
test.count()

root
 |-- sepal_length(cm): double (nullable = true)
 |-- sepal_width(cm): double (nullable = true)
 |-- petal_length(cm): double (nullable = true)
 |-- petal_width(cm): double (nullable = true)
 |-- species: long (nullable = true)

+----------------+---------------+----------------+---------------+-------+
|sepal_length(cm)|sepal_width(cm)|petal_length(cm)|petal_width(cm)|species|
+----------------+---------------+----------------+---------------+-------+
|             5.1|            3.5|             1.4|            0.2|      0|
|             4.9|            3.0|             1.4|            0.2|      0|
|             4.7|            3.2|             1.3|            0.2|      0|
|             4.6|            3.1|             1.5|            0.2|      0|
|             5.0|            3.6|             1.4|            0.2|      0|
+----------------+---------------+----------------+---------------+-------+
only showing top 5 rows



41

In [15]:
from pyspark.ml.classification import LogisticRegression

from pyspark.ml import Pipeline

assembler = VectorAssembler(inputCols=["sepal_length(cm)", \
                                       "sepal_width(cm)", \
                                       "petal_length(cm)", \
                                       "petal_width(cm)"], 
                            outputCol="features")

lr = LogisticRegression(featuresCol='features', labelCol='species')

pipeline = Pipeline(stages=[assembler, lr])

pipeline_model = pipeline.fit(train)

print(pipeline_model)

PipelineModel_4bc0bfdd3457eb10f265


In [16]:
lr_model = pipeline_model.stages[1]
print(lr_model.numClasses)

3


In [17]:
train.schema

StructType(List(StructField(sepal_length(cm),DoubleType,true),StructField(sepal_width(cm),DoubleType,true),StructField(petal_length(cm),DoubleType,true),StructField(petal_width(cm),DoubleType,true),StructField(species,LongType,true)))

In [18]:
prediction_lr = pipeline_model.transform(test)

score_lr = prediction_lr.select(['species', 'prediction'])
score_lr.show(n=score_lr.count())

prediction_lr.show(5)

+-------+----------+
|species|prediction|
+-------+----------+
|      0|       0.0|
|      0|       0.0|
|      1|       1.0|
|      1|       1.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      0|       0.0|
|      1|       1.0|
|      1|       1.0|
|      1|       1.0|
|      1|       1.0|
|      2|       2.0|
|      1|       1.0|
|      1|       1.0|
|      2|       2.0|
|      1|       1.0|
|      2|       2.0|
|      1|       1.0|
|      1|       1.0|
|      2|       2.0|
|      1|       1.0|
|      1|       1.0|
|      2|       2.0|
|      1|       1.0|
|      1|       1.0|
|      1|       1.0|
|      2|       2.0|
|      1|       1.0|
|      1|       2.0|
|      2|       2.0|
|      2|       2.0|
|      1|       1.0|
|      2|       2.0|
|      2|       2.0|
|      2|       2.0|
|      2|       2.0|
|      2|       2.0|
|      2|       2.0|
+-------+----------+

+----------------+---------------+----------------+---

In [19]:
from jpmml import toPMMLBytes

In [21]:
pmmlBytes = toPMMLBytes(sc, train, pipeline_model)
print(pmmlBytes.decode("UTF-8"))

Py4JJavaError: An error occurred while calling z:org.jpmml.sparkml.ConverterUtil.toPMMLByteArray.
: java.lang.NoSuchMethodError: org.dmg.pmml.OutputField.<init>(Lorg/dmg/pmml/FieldName;Lorg/dmg/pmml/DataType;)V
	at org.jpmml.converter.ModelUtil.createPredictedField(ModelUtil.java:194)
	at org.jpmml.sparkml.ClassificationModelConverter.encodeOutput(ClassificationModelConverter.java:76)
	at org.jpmml.sparkml.ModelConverter.registerModel(ModelConverter.java:165)
	at org.jpmml.sparkml.ConverterUtil.toPMML(ConverterUtil.java:76)
	at org.jpmml.sparkml.ConverterUtil.toPMMLByteArray(ConverterUtil.java:132)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.inv